# Blondy's Soaring Prediction System

In [1]:
import json
import os
import shutil
from herbie import Herbie
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from herbie.toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd
import matplotlib as mpl

pd.set_option('display.max_rows', 500)

recent = pd.Timestamp.utcnow().floor("6h").tz_localize(None) #- pd.Timedelta("6h")
recent

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ WARNING: Unable to create config file               │
 │      /Users/andrew/.config/herbie/config.toml        │
 │ Herbie will use standard default settings.           │
 │ Consider setting env variable HERBIE_CONFIG_PATH.    │
 ╰──────────────────────────────────────────────────────╯



Timestamp('2024-12-09 00:00:00')

In [2]:
data_dir = "/Users/andrew/data/hrdps/"
#fig_dir = "/Users/andrew/Repos/BMWF/plots/"


In [3]:
# read in the config
with open('bsp_config.json', 'r') as f:
    config = json.load(f)

1) set up templates for each location

2) grab a slice, punch out the locations, merge into template

3) create a time-series for each level at a location, then concatenate the levels to generate a column model

4) save each location dataset, purge the raw data directory

In [4]:
# make a list of the variables we need and at which levels
df = pd.DataFrame(columns=["variable", "level"])

for variable in config["variables"]:
    for level in config["levels"][config["variables"][variable]]:
        _df = pd.DataFrame({"variable":[variable], "level":[level]})
        df = pd.concat([df, _df]) 
        
df = df.reset_index(drop=True)

# get indices of each variable
list(df[df["variable"] == "TMP"].index)
df

,variable,level
0,TCDC,Sfc
1,HPBL,Sfc
2,PRES,Sfc
3,HGT,ISBL_0550
4,HGT,ISBL_0600
5,HGT,ISBL_0650
6,HGT,ISBL_0700
7,HGT,ISBL_0750
8,HGT,ISBL_0800
9,HGT,ISBL_0850


In [25]:
# set up templates for each location
locations = {loc: [None]*len(df) for loc in config["locations"].keys()}
#locations = {loc: np.zeros(len(df)) for loc in config["locations"].keys()} # can this be vectorized?

In [15]:
def get_timestep(i):
    """
    gets variables specified in config for timestep i, dumps them into "locations" 
    dict at the same index i
    """
    # grab a slice
    ds = Herbie(recent,
                model="hrdps",
                fxx=0,
                product="continental/2.5km",
                variable=df["variable"].iloc[i],
                level=df["level"].iloc[i]
               ).xarray()

    # if the variable name is unknown, assign it
    if "unknown" in ds:
        ds = ds.rename({"unknown":df["variable"].iloc[i]})

    # punch out locations
    for loc in config["locations"]:
        locations[loc][i] = ds.sel((((ds.longitude - config["locations"][loc]["longitude"]) ** 2 
                            + (ds.latitude - config["locations"][loc]["latitude"]) ** 2) ** 0.5).argmin(dim=["x", "y"]))

    # purge the data directory
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    
    return None

In [16]:
# do the big loop to get the data
[get_timestep(i) for i in range(len(df))]

✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ msc ┊ IDX @ None


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241209]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241209]


...

✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241209]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F00 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241209]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [37]:
list(df[df["level"] != "Sfc"].index)

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58]

In [20]:
locations["revelstoke"][10]

<xarray.Dataset> Size: 60B
Dimensions:              ()
Coordinates:
    time                 datetime64[ns] 8B 2024-12-09
    step                 timedelta64[ns] 8B 00:00:00
    isobaricInhPa        float64 8B 875.0
    latitude             float64 8B 50.96
    longitude            float64 8B -118.1
    valid_time           datetime64[ns] 8B ...
    gribfile_projection  object 8B None
Data variables:
    gh                   float32 4B ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal
    model:                   hrdps
    product:                 continental/2.5km
    description:             Canada's High Resolution Deterministic Predictio...
    remote_grib:             /Users/andrew/data/hrdps/20241209/20241209T00Z_M...
    local_grib:              /Users/andrew/data/hrdps/20241209/20241209T00Z_M...
    search:                  None

In [8]:
[variable for variable in config["variables"] if config["variables"][variable] == "ISBL"]

['HGT', 'TMP', 'WIND', 'WDIR']

In [9]:
_ds = Herbie(recent,
             model="hrdps",
             fxx=1,
             product="continental/2.5km",
             variable="HGT",
             level="ISBL_0500"
            ).xarray()

✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Dec-09 00:00 UTC F01 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241209]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


In [10]:
_ds

<xarray.Dataset> Size: 66MB
Dimensions:              (y: 1290, x: 2540)
Coordinates:
    time                 datetime64[ns] 8B 2024-12-09
    step                 timedelta64[ns] 8B 01:00:00
    isobaricInhPa        float64 8B 500.0
    latitude             (y, x) float64 26MB ...
    longitude            (y, x) float64 26MB ...
    valid_time           datetime64[ns] 8B ...
    gribfile_projection  object 8B None
Dimensions without coordinates: y, x
Data variables:
    gh                   (y, x) float32 13MB ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal
    model:                   hrdps
    product:                 continental/2.5km
    description:             Canada's High Resolution Deterministic Predictio...
    remote_grib:             /Users/andrew/data/hrdps/20241209/20241209T00Z_M...
    local_grib:              /Users/andrew/data/hrdps/20241209/20241209T00Z_M...
    search:                  None

In [11]:
df.reset_index(drop=True)

,variable,level
0,TCDC,Sfc
1,HPBL,Sfc
2,PRES,Sfc
3,HGT,ISBL_0550
4,HGT,ISBL_0600
5,HGT,ISBL_0650
6,HGT,ISBL_0700
7,HGT,ISBL_0750
8,HGT,ISBL_0800
9,HGT,ISBL_0850
